In [2]:
import pandas as pd
import pymssql

In [3]:
import json
with open('my.json') as file:
    data = json.load(file)
user = data['user']
pwd =  data['password']

In [4]:
conn=pymssql.connect(host='103.24.200.157',user=user,password=pwd,database='InnoculateNCRFISUBI')

In [5]:
events=pd.read_sql("select EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus from lbtEvents where OpenedOn>='2021-06-01 00:00:00' and OpenedOn<'2021-07-01 00:00:00' ",conn)

In [6]:
pd.set_option('display.max_columns',None)
events.shape

(107423, 9)

In [7]:
sitemas=pd.read_sql('Select s_Id,s_Name,McuId from lbtSiteMas',conn)
sitemas.head()

,s_Id,s_Name,McuId
0,11001,NJUL7061,110212
1,11386,NSAM7081,110270
2,11223,NBAN7561,110332
3,11002,NJUL70711111111,110920
4,11192,NSIL7111,110941


In [8]:
merge1=pd.merge(events,sitemas,on='McuId')

In [9]:
merge1.shape

(107423, 11)

In [10]:
remarks=pd.read_sql("select EventId,UpdatedBy,UpdatedON,Remarks from lbtEventsRemarks where UpdatedON>='2021-06-01 00:00:00' and UpdatedON<='2021-07-01 00:00:00' ",conn)
remarks.head()

,EventId,UpdatedBy,UpdatedON,Remarks
0,598514,sowmiya.ganesan@aparinnosys.com,2021-06-05 10:07:34.343,<br/> sowmiya.ganesan@aparinnosys.com Working
1,598619,sowmiya.ganesan@aparinnosys.com,2021-06-05 10:19:10.957,<br/> sowmiya.ganesan@aparinnosys.com Working
2,598682,sowmiya.ganesan@aparinnosys.com,2021-06-05 10:28:07.367,<br/> sowmiya.ganesan@aparinnosys.com Working
3,615596,abilash.srinivasalu@aparinnosys.com,2021-06-09 19:41:06.280,<br/> abilash.srinivasalu@aparinnosys.com working
4,615597,abilash.srinivasalu@aparinnosys.com,2021-06-09 19:42:03.797,<br/> abilash.srinivasalu@aparinnosys.com working


In [11]:
merge2=pd.merge(merge1,remarks,on='EventId')

In [12]:
merge2.shape

(11578, 14)

In [13]:
merge3=merge2.drop_duplicates(keep='first')
merge3['qStatus']=merge3['qStatus'].astype(str)
merge3['qStatus']=merge3['qStatus'].str.lower()
merge4=merge3[merge3['qStatus']=='true']
merge4.shape

(11578, 14)

In [14]:
merge4['ClosedBy']=merge4['ClosedBy'].str.lower()
mergedrop=merge4[merge4['ClosedBy']=='system']
print(mergedrop.shape)
merge5=merge4.drop(mergedrop.index)
print(merge5.shape)

(2, 14)
(11576, 14)


In [15]:
merge5['Remarks']=merge5['Remarks'].str.lower()
merge55=merge5[merge5['Remarks'].str.contains('working')]
merge5drop=merge55[merge55['Remarks'].str.contains('closed')]
merge5drop.shape

(491, 14)

In [16]:
pd.set_option('display.max_rows',None)
Events=merge55.drop(merge5drop.index)
Events.shape

(4489, 14)

In [17]:
Events.head()

,EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus,s_Id,s_Name,UpdatedBy,UpdatedON,Remarks
1,581000,111620,2021-06-01 00:20:45,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 00:20:08.560,yoganathan.munival@aparinnosys.com,2021-06-01 00:29:54.760,true,11115,NPAT7121,yoganathan.munival@aparinnosys.com,2021-06-01 00:21:41.160,<br/> yoganathan.munival@aparinnosys.com worki...
3,581005,111620,2021-06-01 00:32:51,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 00:32:14.533,yoganathan.munival@aparinnosys.com,2021-06-01 00:58:47.500,true,11115,NPAT7121,yoganathan.munival@aparinnosys.com,2021-06-01 00:33:09.750,<br/> yoganathan.munival@aparinnosys.com worki...
9,582651,111620,2021-06-01 11:58:43,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 11:58:41.300,sowmiya.ganesan@aparinnosys.com,2021-06-01 12:14:35.090,true,11115,NPAT7121,sowmiya.ganesan@aparinnosys.com,2021-06-01 12:04:45.767,<br/> sowmiya.ganesan@aparinnosys.com working
12,583341,111620,2021-06-01 13:59:30,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 13:59:28.007,joel.jebaraj@aparinnosys.com,2021-06-01 14:07:18.483,true,11115,NPAT7121,joel.jebaraj@aparinnosys.com,2021-06-01 14:05:52.570,<br/> joel.jebaraj@aparinnosys.com working
14,583946,111620,2021-06-01 16:23:15,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 16:23:12.573,joel.jebaraj@aparinnosys.com,2021-06-01 16:42:52.283,true,11115,NPAT7121,joel.jebaraj@aparinnosys.com,2021-06-01 16:24:05.390,<br/> joel.jebaraj@aparinnosys.com working


In [18]:
Events['TicketAge']=Events['ClosedOn']-Events['OpenedOn']
Events['AcknowledgeAge']=Events['UpdatedON']-Events['OpenedOn']
Events['TicketAge']=Events['TicketAge'].astype('timedelta64[m]')
Events['AcknowledgeAge']=Events['AcknowledgeAge'].astype('timedelta64[m]')

In [19]:
Events=Events.drop_duplicates(keep='first')
Events.shape

(4489, 16)

In [20]:
import numpy as np

In [21]:
conditions = [
    (Events['EventType'] == 'SOS') & (Events['TicketAge'] > 30),
    (Events['EventType'] == 'Critical') & (Events['TicketAge'] > 60),
    (Events['EventType'] == 'SOS') & (Events['AcknowledgeAge'] > 5),
    (Events['EventType'] == 'Critical') & (Events['AcknowledgeAge'] > 10)]

# create a list of the values we want to assign for each condition
values = ['Penalty Applicable','Penalty Applicable','Penalty Applicable','Penalty Applicable']

# create a new column and use np.select to assign values to it using our lists as arguments
Events['Penalty'] = np.select(conditions, values)

# display updated DataFrame
Events.head()

,EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus,s_Id,s_Name,UpdatedBy,UpdatedON,Remarks,TicketAge,AcknowledgeAge,Penalty
1,581000,111620,2021-06-01 00:20:45,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 00:20:08.560,yoganathan.munival@aparinnosys.com,2021-06-01 00:29:54.760,true,11115,NPAT7121,yoganathan.munival@aparinnosys.com,2021-06-01 00:21:41.160,<br/> yoganathan.munival@aparinnosys.com worki...,9.0,1.0,0
3,581005,111620,2021-06-01 00:32:51,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 00:32:14.533,yoganathan.munival@aparinnosys.com,2021-06-01 00:58:47.500,true,11115,NPAT7121,yoganathan.munival@aparinnosys.com,2021-06-01 00:33:09.750,<br/> yoganathan.munival@aparinnosys.com worki...,26.0,0.0,0
9,582651,111620,2021-06-01 11:58:43,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 11:58:41.300,sowmiya.ganesan@aparinnosys.com,2021-06-01 12:14:35.090,true,11115,NPAT7121,sowmiya.ganesan@aparinnosys.com,2021-06-01 12:04:45.767,<br/> sowmiya.ganesan@aparinnosys.com working,15.0,6.0,Penalty Applicable
12,583341,111620,2021-06-01 13:59:30,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 13:59:28.007,joel.jebaraj@aparinnosys.com,2021-06-01 14:07:18.483,true,11115,NPAT7121,joel.jebaraj@aparinnosys.com,2021-06-01 14:05:52.570,<br/> joel.jebaraj@aparinnosys.com working,7.0,6.0,Penalty Applicable
14,583946,111620,2021-06-01 16:23:15,ATM-1 Battery Bank Theft Attempt,SOS,2021-06-01 16:23:12.573,joel.jebaraj@aparinnosys.com,2021-06-01 16:42:52.283,true,11115,NPAT7121,joel.jebaraj@aparinnosys.com,2021-06-01 16:24:05.390,<br/> joel.jebaraj@aparinnosys.com working,19.0,0.0,0


In [22]:
Events['Penalty'].value_counts()

0                     2411
Penalty Applicable    2078
Name: Penalty, dtype: int64

In [23]:
sos=Events[(Events['EventType']=='SOS') & (Events['Penalty']=='Penalty Applicable')]

In [24]:
sos.shape

(944, 17)

In [28]:
penalty=Events[Events['Penalty']=='Penalty Applicable']

In [29]:
penalty.to_csv('FISUBIJunepenalty.csv')